# Imputing land values to the FRS

In [12]:
import plotly.express as px
from ubicenter import format_fig
import numpy as np
import pandas as pd
import microdf as mdf
import synthimpute as si
import sklearn

In [24]:
was.dvtotgirR6

0        32600.0
1        40300.0
2        17900.0
3        73100.0
4        34200.0
          ...   
18024    56700.0
18025    28200.0
18026    67300.0
18027    24700.0
18028    63500.0
Name: dvtotgirR6, Length: 18029, dtype: float64

In [25]:
RENAMES = {
    "dvtotgirR6": "gross_income"
}

was = pd.read_csv("~/was_round_6_hhold_eul_mar_20.tab", delimiter="\t")
was = was.replace(" ", 0).astype(float).rename(columns=RENAMES)

WAS_WEIGHT = "R6xshhwgt"

# train, test = sklearn.model_selection.train_test_split(df)

In [15]:
mdf.weighted_sum(df, "TotWlthR6", WAS_WEIGHT) / 1e9

14160.955575659307

In [4]:
mdf.weighted_sum(df, "DVLUKValR6_sum", "R6xshhwgt") / 1e9

63.004871507828334

In [5]:
mdf.weighted_sum(df, "DVLOSValR6_sum", "R6xshhwgt") / 1e9

84.2186628588207

In [17]:
TRAIN_COLS = [
    "gross_income",  # Household gross income.
    # "DVTotGIRW5",  # Household net income.
]

IMPUTE_COLS = [
    "TotWlthR6",  # Total wealth.
]

In [44]:
test["pred_wealth"] = si.rf_impute(
    x_train=train[TRAIN_COLS],
    y_train=train[IMPUTE_COLS],
    x_new=test[TRAIN_COLS],
    sample_weight_train=train[WEIGHT],
)

/home/mghenis/anaconda3/lib/python3.7/site-packages/synthimpute/rf_impute.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train, sample_weight=sample_weight_train)
/home/mghenis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [46]:
test[TRAIN_COLS + ["pred_wealth"]]

,DVTotNIRW5,pred_wealth
10454,17200.0,1.524809e+05
11367,14000.0,2.516482e+05
7552,56100.0,5.508292e+05
17891,87400.0,1.191353e+06
5883,38500.0,5.762272e+05
...,...,...
10066,32000.0,5.526614e+05
18802,7680.0,1.037723e+04
18006,47900.0,6.614380e+05
13934,27400.0,2.970193e+05


In [32]:
from openfisca_uk import Microsimulation

sim = Microsimulation(year=2020)

In [33]:
frs = sim.df(["gross_income", "num_adults"], map_to="household")

In [19]:
was.gross_income


AttributeError: 'DataFrame' object has no attribute 'gross_income'

In [26]:
frs["pred_wealth"] = si.rf_impute(
    x_train=was[TRAIN_COLS],
    y_train=was[IMPUTE_COLS],
    x_new=frs[TRAIN_COLS],
    sample_weight_train=was[WAS_WEIGHT],
)

In [31]:
mdf.weighted_sum(frs, "pred_wealth", "weight") / 1e9

16508.52899089139